In [22]:
query = """
SELECT DISTINCT ?person ?personLabel ?personDescription ?sitelinks ?genderLabel ?dob ?birthPlaceLabel ?coords ?image
WHERE {
    ?person wdt:P31 wd:Q5;            # Any instance of a human
          wdt:P19/wdt:P131* wd:Q60; # Who was born in any value (eg. a hospital)
          wikibase:sitelinks ?sitelinks.        
          ?person wdt:P21 ?gender.
          ?person wdt:P569 ?dob.
          ?person wdt:P106 ?occupation.
          ?person wdt:P551 ?residence.
          ?person wdt:P19 ?birthPlace.
          ?birthPlace wdt:P625 ?coords.
          ?person wdt:P18 ?image.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY DESC(?sitelinks)
LIMIT 2000
"""

In [2]:
import sys
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

class WikiDataQueryResults:
    """
    A class that can be used to query data from Wikidata using SPARQL and return the results as a Pandas DataFrame or a list
    of values for a specific key.
    """
    def __init__(self, query: str):
        """
        Initializes the WikiDataQueryResults object with a SPARQL query string.
        :param query: A SPARQL query string.
        """
        self.user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        self.endpoint_url = "https://query.wikidata.org/sparql"
        self.sparql = SPARQLWrapper(self.endpoint_url, agent=self.user_agent)
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

    def __transform2dicts(self, results: List[Dict]) -> List[Dict]:
        """
        Helper function to transform SPARQL query results into a list of dictionaries.
        :param results: A list of query results returned by SPARQLWrapper.
        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results

    def _load(self) -> List[Dict]:
        """
        Helper function that loads the data from Wikidata using the SPARQLWrapper library, and transforms the results into
        a list of dictionaries.
        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        results = self.sparql.queryAndConvert()['results']['bindings']
        results = self.__transform2dicts(results)
        return results

    def load_as_dataframe(self) -> pd.DataFrame:
        """
        Executes the SPARQL query and returns the results as a Pandas DataFrame.
        :return: A Pandas DataFrame representing the query results.
        """
        results = self._load()
        return pd.DataFrame.from_dict(results)

In [23]:
data_extracter = WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
print(df.head())

                                  person         personLabel  \
0  http://www.wikidata.org/entity/Q22686        Donald Trump   
1   http://www.wikidata.org/entity/Q4617         Corbin Bleu   
2  http://www.wikidata.org/entity/Q19848           Lady Gaga   
3  http://www.wikidata.org/entity/Q33866  Theodore Roosevelt   
4  http://www.wikidata.org/entity/Q39246     Richard Feynman   

                                   personDescription sitelinks genderLabel  \
0  President-elect of the United States; 45th pre...       311        male   
1  American actor, model, dancer, film producer a...       218        male   
2  American singer, songwriter, and actress (born...       188      female   
3  26th President of the United States from 1901 ...       188        male   
4         American theoretical physicist (1918–1988)       152        male   

                    dob                  birthPlaceLabel  \
0  1946-06-14T00:00:00Z  Jamaica Hospital Medical Center   
1  1989-02-21T00:00:00Z   

In [17]:
for index, row in df.iterrows():
  print(row)

person                           http://www.wikidata.org/entity/Q22686
personLabel                                               Donald Trump
personDescription    President-elect of the United States; 45th pre...
sitelinks                                                          311
genderLabel                                                       male
dob                                               1946-06-14T00:00:00Z
image                http://commons.wikimedia.org/wiki/Special:File...
Name: 0, dtype: object
person                            http://www.wikidata.org/entity/Q4617
personLabel                                                Corbin Bleu
personDescription    American actor, model, dancer, film producer a...
sitelinks                                                          218
genderLabel                                                       male
dob                                               1989-02-21T00:00:00Z
image                http://commons.wikimedia.org/wiki